<a href="https://colab.research.google.com/github/crypto-saac/sequence-to-sequence-architectures-of-character-level-machine-translation-/blob/main/Character_Level_recurrent_Seq_2_Seq_Model_LayeredLSTM_LSTM_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N']

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens (num_encoder_tokenes):", num_encoder_tokens)
print("Number of unique output tokens (num_decoder_tokens):", num_decoder_tokens)
print("Max sequence length for inputs (max_encoder_seq_length):", max_encoder_seq_length)
print("Max sequence length for outputs (max_decoder_seq_length):", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens (num_encoder_tokenes): 70
Number of unique output tokens (num_decoder_tokens): 93
Max sequence length for inputs (max_encoder_seq_length): 14
Max sequence length for outputs (max_decoder_seq_length): 59


In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

# Define the first LSTM layer
encoder_lstm_1 = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs_1, state_h_1, state_c_1 = encoder_lstm_1(encoder_inputs)

# Define the second LSTM layer
encoder_lstm_2 = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h_2, state_c_2 = encoder_lstm_2(encoder_outputs_1)

# Concatenate the forward and backward hidden states
state_h = keras.layers.Concatenate()([state_h_2, state_c_2])
state_c = keras.layers.Concatenate()([state_h_2, state_c_2])

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim*2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)



In [ ]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 12s 33ms/step - loss: 1.2159 - accuracy: 0.7313 - val_loss: 1.0189 - val_accuracy: 0.7140
Epoch 2/100
125/125 [==============================] - 3s 21ms/step - loss: 0.8238 - accuracy: 0.7732 - val_loss: 0.8152 - val_accuracy: 0.7839
Epoch 3/100
125/125 [==============================] - 3s 24ms/step - loss: 0.6651 - accuracy: 0.8146 - val_loss: 0.6845 - val_accuracy: 0.8042
Epoch 4/100
125/125 [==============================] - 3s 21ms/step - loss: 0.5791 - accuracy: 0.8323 - val_loss: 0.6083 - val_accuracy: 0.8233
Epoch 5/100
125/125 [==============================] - 3s 21ms/step - loss: 0.5313 - accuracy: 0.8448 - val_loss: 0.5746 - val_accuracy: 0.8319
Epoch 6/100
125/125 [==============================] - 3s 21ms/step - loss: 0.4980 - accuracy: 0.8544 - val_loss: 0.5470 - val_accuracy: 0.8407
Epoch 7/100
125/125 [==============================] - 3s 21ms/step - loss: 0.4710 - accuracy: 0.8613 - val_loss: 0.5300 - val_accuracy

In [ ]:
pip install sacrebleu


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

predictions = model.predict([encoder_input_data, decoder_input_data])

# Convert predictions to text
predicted_texts = []
for prediction in predictions:
    predicted_text = ""
    for timestep in prediction:
        # Get index of highest probability
        char_index = np.argmax(timestep)
        # Convert index to character
        char = target_characters[char_index]
        # Append character to predicted text
        predicted_text += char
        # Stop if end sequence character is generated
        if char == "\n":
            break
    predicted_texts.append(predicted_text)

# Convert target texts to list of lists
target_texts_list = [list(text) for text in target_texts]

# Convert predicted texts to list of lists
predicted_texts_list = [list(text) for text in predicted_texts]

# Calculate BLEU score
bleu_score = corpus_bleu([[text] for text in target_texts_list], predicted_texts_list)
print("2 layered LSTM:")
print("BLEU score:", bleu_score)

313/313 [==============================] - 2s 7ms/step
2 layered LSTM:
BLEU score: 0.775977632509008
